In [10]:
# We import all the libraries for
#numpy,pandas,sklearn,matplotlib are the primary libraries

import numpy as np
import pandas as pd
import sys
from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import ExtraTreesClassifier
#from feature_vector import gen_test, gen_train
from sklearn.feature_selection import VarianceThreshold
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPRegressor
from IPython.display import clear_output
from sklearn.model_selection import train_test_split
from IPython.display import clear_output
clear_output()
#svm and with cross validation
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from IPython.display import clear_output
from sklearn.inspection import permutation_importance
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
import warnings
import matplotlib.pyplot as plt
from IPython.display import clear_output
from sklearn import preprocessing
warnings.filterwarnings("ignore")
clear_output()


In [18]:
def svm_lanth(A,B):
    
    # 80% test train split
    x_train, x_test, y_train, y_test = train_test_split(
                                       A, B,test_size=0.2)
    rows=np.shape(x_test)[0]
    cols=np.shape(y_test)[1]
    ytest_clf=np.zeros((rows,cols))
    
    # Tuning parameters: estimator__gamma,estimator__C
    tuned_parameters = [  {"estimator__kernel": ["rbf"], "estimator__gamma": [1e-3, 1e-4], "estimator__C": [1, 10, 100, 1000]}]

    
    ## Defining the regressor for SVR: For multiple output
    mor=MultiOutputRegressor(SVR())
    
    #Gridsearching to get the best parameter(s) with 5-fold cross validation
    clf=GridSearchCV(mor,tuned_parameters,scoring="neg_mean_squared_error",cv=5).fit(x_train, y_train)
    
    # Getting the best parameters. estimator__gamma in this case
    best_param=clf.cv_results_['params'][clf.best_index_]
    
    #Training 
    nmor=MultiOutputRegressor(SVR(kernel='rbf',C=best_param['estimator__C'],gamma=best_param['estimator__gamma']))
    
    #for prediction
    tlf = nmor.fit(x_train, y_train)
    
    #for permutation importance
    #tlf=nmor.fit(A,B)
    
    
    #Testing with full data A,B for importance 
    #result = permutation_importance(tlf, x_train, y_train, n_repeats=5)
    
    # Prediction
    ytest_clf=tlf.predict(x_test)
    
    #returning squared error
    return(np.mean((ytest_clf-y_test)**2)) 

def kernel_rid_lanth(A,B):
    
    # 80% test train split
    x_train, x_test, y_train, y_test = train_test_split(
                                       A, B,test_size=0.2)
    rows=np.shape(x_test)[0]
    cols=np.shape(y_test)[1]
    ytest_clf=np.zeros((rows,cols))
    
    
     # Tuning parameters: estimator__alpha,estimator__kernel,estimator__gamma
    tuned_parameters = {"estimator__alpha":[0.1,0.001,0.00001],"estimator__kernel":["rbf","linear","polynomial"],
            "estimator__gamma": [1e-3, 1e-4]}

    
    ## Defining the regressor for Kernel Ridge : Multiple output
    mor=MultiOutputRegressor(KernelRidge())
    clf=GridSearchCV(mor,tuned_parameters,scoring="neg_mean_squared_error",cv=5).fit(x_train, y_train) #fit(A,B)#.fit(x_train, y_train)

    # Getting the best parameters. estimator__gamma in this case
    best_param=clf.cv_results_['params'][clf.best_index_]
    nmor=MultiOutputRegressor(KernelRidge(kernel=best_param['estimator__kernel'],alpha=best_param['estimator__alpha'],gamma=best_param['estimator__gamma']))
    
    #Testing with full data A,B for importance
    #tlf=nmor.fit(A,B)
    #result = permutation_importance(tlf, A,B, n_repeats=5)
    
      
    #Training with test-train
    tlf=nmor.fit(x_train, y_train)
    #Prediction
    ytest_clf=tlf.predict(x_test)
    
    #Squared error
    return(np.mean((ytest_clf-y_test)**2))

    #return(result)

def neural_net_lanth_2(A,B):
    # 80% test train split
    x_train, x_test, y_train, y_test = train_test_split(
                                       A, B,test_size=0.2)
    rows=np.shape(x_test)[0]
    cols=np.shape(y_test)[1]
    ytest_clf=np.zeros((rows,cols))
    
     # Tuning parameters: estimator__hidden_layer_sizes
    tuned_parameters = {"estimator__activation": ["logistic"],
               "estimator__solver":["lbfgs"],
               "estimator__hidden_layer_sizes":[10,20,30,40,50],
               "estimator__alpha":[0.001], "estimator__learning_rate":["constant"] }

    ## Defining the regressor for Neural Network : Multiple output
    mor= MultiOutputRegressor(MLPRegressor())
    clf=GridSearchCV(mor, tuned_parameters,scoring="neg_mean_squared_error",cv=2).fit(x_train,y_train)
    
    # Getting the best parameters.
    best_param=clf.cv_results_['params'][clf.best_index_]
    nmor=MultiOutputRegressor(MLPRegressor(alpha=best_param['estimator__alpha'],hidden_layer_sizes=best_param['estimator__hidden_layer_sizes'],activation='identity', solver='lbfgs',learning_rate='constant'))
    tlf =nmor.fit(x_train, y_train)
    
   #Prediction
    ytest_clf=tlf.predict(x_test)
    
    #Testing with full data A,B for importance
    #tlf=nmor.fit(A,B)
    #result = permutation_importance(tlf, A,B, n_repeats=5)
    
    #Squared error
    return(np.mean((ytest_clf-y_test)**2))
    #return(result)
    
def random_forest_lanth(A,B):
    # 80% test train split
    x_train, x_test, y_train, y_test = train_test_split(A, B,test_size=0.2)
    rows=np.shape(x_test)[0]
    cols=np.shape(y_test)[1]
    ytest_clf=np.zeros((rows,cols))
    
     # Tuning parameters:estimator__n_estimators,estimator__max_samples
    tuned_parameters = [  {"estimator__n_estimators": [20,30,40], "estimator__criterion": ["poisson"], "estimator__max_samples": [100, 200]}]

    ## Defining the regressor for Neural Network : Multiple output
    mor=MultiOutputRegressor(RandomForestRegressor())
    clf=GridSearchCV(mor,tuned_parameters,scoring="neg_mean_squared_error",cv=4)#.fit(A,B)#fit(A,B)#
    #clf.fit(A,B.iloc[:])
    clf.fit(x_train,y_train)
    
    # Getting the best parameters.
    best_param=clf.cv_results_['params'][clf.best_index_]
    nmor=MultiOutputRegressor(RandomForestRegressor(n_estimators=best_param['estimator__n_estimators'],max_samples=best_param['estimator__max_samples']))
    tlf =nmor.fit(x_train,y_train)#fit(A,B) #fit(x_train,y_train) #fit(A,B.iloc[:])
    
    #Testing with full data A,B for importance
    #tlf=nmor.fit(A,B)
    #result = permutation_importance(tlf, A,B, n_repeats=5)
    
    #Prediction
    ytest_clf=tlf.predict(x_test)
    
    #Squared error
    return(np.mean((ytest_clf-y_test)**2)) 
    #return(result)    

In [21]:
import matplotlib.pyplot as plt
from IPython.display import clear_output
from sklearn import preprocessing
warnings.filterwarnings("ignore")
clear_output()

# Loading the screened X and Y matrices. 
x_screened=pd.read_csv('/Users/sampreetibhattacharya/Documents/vestalab/ML_proj/X_screen_mapped.csv')
ytr=pd.read_csv('/Users/sampreetibhattacharya/Documents/vestalab/ML_proj/Y.csv')

x_screen_new=pd.DataFrame(np.matrix(x_screened.iloc[:,1:]),columns=x_screened.columns[1:])
ytr_new=pd.DataFrame(np.matrix(ytr.iloc[:,1:]),columns=ytr.columns[1:])

# Defining error vec matrix with 20 runs for each four methods, SVM, Kernel Ridge,
#NeuralNetwork and Random Forest

error_vec=np.zeros((20,4))

# scale the response and the predictors
scaler = preprocessing.MinMaxScaler().fit(ytr_new)
y_scaled=scaler.transform(ytr_new)            
scaler = preprocessing.MinMaxScaler().fit(x_screen_new.iloc[:,:])
new_x_screened=pd.DataFrame(scaler.transform(x_screen_new.iloc[:,:]),columns=x_screen_new.columns)                                                 # huge among each other

#running times for reproducibility
for j in range(0,20):
    error_vec[j,0]=kernel_rid_lanth(new_x_screened,y_scaled)
    error_vec[j,1]=random_forest_lanth(new_x_screened,y_scaled)
    error_vec[j,2]=svm_lanth(new_x_screened,y_scaled) 
    error_vec[j,3]=neural_net_lanth_2(x_screen_new,y_scaled)
    print("completed loop",j)
    

## Storing the errors
pd.DataFrame(error_vec,columns=["Kernel_Ridge","random_forest","svm","neural_net"])


completed loop 0
completed loop 1
completed loop 2
completed loop 3
completed loop 4
completed loop 5
completed loop 6
completed loop 7
completed loop 8
completed loop 9
completed loop 10
completed loop 11
completed loop 12
completed loop 13
completed loop 14
completed loop 15
completed loop 16
completed loop 17
completed loop 18
completed loop 19


,Kernel_Ridge,random_forest,svm,neural_net
0,0.000547,0.000396,0.003049,0.000618
1,0.000454,0.000336,0.002833,0.000660
2,0.000450,0.000547,0.002679,0.000688
3,0.000438,0.000398,0.002513,0.000596
4,0.000450,0.000378,0.002900,0.000650
5,0.000498,0.000399,0.002625,0.000637
6,0.000566,0.000380,0.002661,0.000573
7,0.000499,0.000501,0.002678,0.000827
8,0.000520,0.000576,0.002853,0.000734
9,0.000406,0.000550,0.002706,0.000563


In [27]:
#Finding the pooled mean, in error_vec[0:Kernel Ridge,random forest, svm , neural networj]
error_vec.mean(axis=0)
#Random Forest gives the minimum error

array([0.00046263, 0.00043736, 0.00279209, 0.00065784])

In [31]:
#Choose the model to get the importance mean
# and uncomment lines needed for variable importance.
def random_forest_lanth(A,B):
    # 80% test train split
    #x_train, x_test, y_train, y_test = train_test_split(A, B,test_size=0.2)
#     rows=np.shape(A)[0]
#     cols=np.shape(B)[1]
#     ytest_clf=np.zeros((rows,cols))
    
     # Tuning parameters:estimator__n_estimators,estimator__max_samples
    tuned_parameters = [  {"estimator__n_estimators": [20,30,40], "estimator__criterion": ["poisson"], "estimator__max_samples": [100, 200]}]

    ## Defining the regressor for Neural Network : Multiple output
    mor=MultiOutputRegressor(RandomForestRegressor())
    clf=GridSearchCV(mor,tuned_parameters,scoring="neg_mean_squared_error",cv=4)#.fit(A,B)#fit(A,B)#
    clf.fit(A,B)
    #clf.fit(x_train,y_train)
    
    # Getting the best parameters.
    best_param=clf.cv_results_['params'][clf.best_index_]
    nmor=MultiOutputRegressor(RandomForestRegressor(n_estimators=best_param['estimator__n_estimators'],max_samples=best_param['estimator__max_samples']))
    #tlf =nmor.fit(x_train,y_train)#fit(A,B) #fit(x_train,y_train) #fit(A,B.iloc[:])
    
    #Testing with full data A,B for importance
    tlf=nmor.fit(A,B)
    result = permutation_importance(tlf, A,B, n_repeats=5)
    
    #Prediction
    #ytest_clf=tlf.predict(x_test)
    
    #Squared error
    #return(np.mean((ytest_clf-y_test)**2)) 
    return(result)    

In [32]:
### Permutation importance code for random forest ######

# Permuting columns to ensure robustness
l=np.random.permutation(x_screen_new.columns)
xl=x_screen_new[l] # to permute the columns

## Defining the importance array 
importance_rf=np.zeros((10,100))

for i in range(0,10):
    print("in loop ",i)
    importance=random_forest_lanth(xl,y_scaled)
    importance_rf[i,:]=np.transpose(importance.importances_mean)

importance_rf_final=pd.DataFrame(importance_rf,columns=xl.columns)

in loop  0
in loop  1
in loop  2
in loop  3
in loop  4
in loop  5
in loop  6
in loop  7
in loop  8
in loop  9


In [80]:
indx=importance_rf_final.mean(axis=0).argsort()[::-1]
pd.DataFrame(importance_rf_final.columns[indx],importance_rf_final.mean(axis=0)[indx])


,0
6.524636e-01,Asite_shannon_radii_weighted_avg
6.128266e-01,Bsite_shannon_radii_weighted_avg
3.544867e-01,Bsite_Ionic Radius (angstroms)_weighted_avg
2.901397e-01,shannon_radii_AB_avg
9.395373e-05,Bsite_IsMetal_weighted_avg
...,...
6.951802e-09,Bsite_IsTetragonal_range
-3.270981e-08,host_Asite0_IsPnictide
-4.166642e-08,Bsite_OrbitalS_range
-1.589878e-07,Bsite_IsOrthorhombic_min


In [81]:
def kernel_rid_lanth(A,B):
    

     # Tuning parameters: estimator__alpha,estimator__kernel,estimator__gamma
    tuned_parameters = {"estimator__alpha":[0.1,0.001,0.00001],"estimator__kernel":["rbf","linear","polynomial"],
            "estimator__gamma": [1e-3, 1e-4]}

    
    ## Defining the regressor for Kernel Ridge : Multiple output
    mor=MultiOutputRegressor(KernelRidge())
    clf=GridSearchCV(mor,tuned_parameters,scoring="neg_mean_squared_error",cv=5).fit(A,B)#.fit(x_train, y_train)

    # Getting the best parameters. estimator__gamma in this case
    best_param=clf.cv_results_['params'][clf.best_index_]
    nmor=MultiOutputRegressor(KernelRidge(kernel=best_param['estimator__kernel'],alpha=best_param['estimator__alpha'],gamma=best_param['estimator__gamma']))
    
    #Testing with full data A,B for importance
    tlf=nmor.fit(A,B)
    result = permutation_importance(tlf, A,B, n_repeats=5)
    return(result)


In [82]:
### Permutation importance code for Kernel Ridge ######

# Permuting columns to ensure robustness
l=np.random.permutation(x_screen_new.columns)
xl=x_screen_new[l] # to permute the columns

## Defining the importance array 
importance_krid=np.zeros((10,100))

for i in range(0,10):
    print("in loop ",i)
    importance=kernel_rid_lanth(xl,y_scaled)
    importance_krid[i,:]=np.transpose(importance.importances_mean)

importance_rf_final=pd.DataFrame(importance_krid,columns=xl.columns)
indx=importance_rf_final.mean(axis=0).argsort()[::-1]
pd.DataFrame(importance_rf_final.columns[indx],importance_rf_final.mean(axis=0)[indx])


in loop  0
in loop  1
in loop  2
in loop  3
in loop  4
in loop  5
in loop  6
in loop  7
in loop  8
in loop  9


,0
5.443490e-01,Asite_shannon_radii_weighted_avg
5.343136e-01,Bsite_shannon_radii_weighted_avg
3.720709e-01,Bsite_Ionic Radius (angstroms)_weighted_avg
1.730565e-01,shannon_radii_AB_avg
3.327603e-04,host_Bsite0_IsOrthorhombic
...,...
9.964706e-08,Bsite_IsMetalloid_range
9.964706e-08,Bsite_IsMetalloid_max
9.964706e-08,Bsite_IsMetal_range
3.434546e-08,Bsite_IsTetragonal_max
